<a href="https://colab.research.google.com/github/kailashSwaminathan/cce_dl4ai_2023/blob/%E0%A4%B8%E0%A4%BE%E0%A4%97%E0%A4%B0%E0%A4%82/notebooks/naive_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **A NAIVE IMPLEMENTATION OF NEURAL NETWORK USING TENSORFLOW**

---





In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras import Input
from keras.layers import Dense, Flatten
from keras.utils import plot_model
from keras.datasets import mnist

Implementation of a Dense Layer class

In [ ]:
class NaiveDenseLayer:
    def __init__(self, input_size, output_size, activation): 
        self.activation = activation
        w_shape = (input_size, output_size)
        w_initial = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        # For updation the type should be tf.Variable
        self.weights = tf.Variable(w_initial)
        b_shape = (output_size,)
        b_initial = tf.zeros(b_shape)
        self.biases = tf.Variable(b_initial)

    def __call__(self, inputs):
        # self.biases is broadcasted
        return self.activation(tf.matmul(inputs, self.weights) + self.biases) 

    @property
    def weights(self):
        return (self.weights, self.biases)

In [ ]:
class ForwardPass:
    """ Forward pass of the input through
        the Neural Network Layers
    """
    def __init__(self, layers):
        """ layers: list
        """
        self.layers = layers

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x

    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

        

In [ ]:
model = ForwardPass([
    NaiveDenseLayer(input_size=28*28, output_size=512, activation=tf.nn.relu),
    NaiveDenseLayer(input_size=512, output_size=10, activation=tf.nn.softmax)
])

In [ ]:
(train_images, train_labels),(test_images, test_labels) = mnist.load_data()

In [ ]:
train_images = train_images.reshape((len(train_images), 28*28)).astype("float32")/255
test_images = test_images.reshape((len(test_images), 28*28)).astype("float32")/255

In [ ]:
plt.figure(figsize=(10,10))
for i in range(20):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i].reshape((28,28)), cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])

In [ ]:
class BatchGenerator:
    """ Generates a batch
    """
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self._num_batches = math.ceil(len(images)/batch_size)

    def next(self):
        images = self.images[self.index : (self.index + self.batch_size)]
        labels = self.labels[self.index : (self.index + self.batch_size)]
        self.index += self.batch_size
        return images, labels

    @property
    def num_batches(self):
        return self._num_batches

In [ ]:
learning_rate = 1e-3
def update_weights(gradients, weights):
    for g,w in zip(gradients, weights):
        w.assign_sib(g*learning_rate)  # w -= g*lr


def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        per_sample_losses = keras.losses.sparse_categorical_crossentropy( labels_batch, predictions )
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

def fit(model, images, labels, epochs, batch_size=128):
    for epoch in range(epochs):
        print(f"Epoch: {epoch}")
        batch_generator = BatchGenerator(images, labels)
        for batch_count in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"Loss at batch({batch_counter}): {loss:.2f}")

In [ ]:
fit(model, train_images, train_labels, epochs=10, batch_size=128)